In [1]:
import pandas as pd
import requests
import sys
import sqlite3


In [2]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


# Aprendiendo pandas 

Queremos obtener una informaciòn sencilla del conjunto de los datos 

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14585 entries, 0 to 14584
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   a_o                          14585 non-null  object
 1   c_digo_municipio             14585 non-null  object
 2   municipio                    14585 non-null  object
 3   c_digo_departamento          14585 non-null  object
 4   departamento                 14585 non-null  object
 5   c_digo_etc                   14585 non-null  object
 6   etc                          14585 non-null  object
 7   poblaci_n_5_16               14579 non-null  object
 8   tasa_matriculaci_n_5_16      14470 non-null  object
 9   cobertura_neta               14474 non-null  object
 10  cobertura_neta_transici_n    14533 non-null  object
 11  cobertura_neta_primaria      14494 non-null  object
 12  cobertura_neta_secundaria    14491 non-null  object
 13  cobertura_neta_media         14

In [4]:
df_raw.columns.to_list()

['a_o',
 'c_digo_municipio',
 'municipio',
 'c_digo_departamento',
 'departamento',
 'c_digo_etc',
 'etc',
 'poblaci_n_5_16',
 'tasa_matriculaci_n_5_16',
 'cobertura_neta',
 'cobertura_neta_transici_n',
 'cobertura_neta_primaria',
 'cobertura_neta_secundaria',
 'cobertura_neta_media',
 'cobertura_bruta',
 'cobertura_bruta_transici_n',
 'cobertura_bruta_primaria',
 'cobertura_bruta_secundaria',
 'cobertura_bruta_media',
 'deserci_n',
 'deserci_n_transici_n',
 'deserci_n_primaria',
 'deserci_n_secundaria',
 'deserci_n_media',
 'aprobaci_n',
 'aprobaci_n_transici_n',
 'aprobaci_n_primaria',
 'aprobaci_n_secundaria',
 'aprobaci_n_media',
 'reprobaci_n',
 'reprobaci_n_transici_n',
 'reprobaci_n_primaria',
 'reprobaci_n_secundaria',
 'reprobaci_n_media',
 'repitencia',
 'repitencia_transici_n',
 'repitencia_primaria',
 'repitencia_secundaria',
 'repitencia_media',
 'tama_o_promedio_de_grupo',
 'sedes_conectadas_a_internet']

Generemos una "muestra" del cojunto de datos 

1. tomando los primeros registros del conjunto de datos 

In [5]:
df_raw.head()

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [6]:
pd.set_option('display.max_columns', None) ## muestra todos los datos

In [9]:
df_raw.iloc[0:5, 0:10]  # Muestra las primeras 5 filas y 10 columnas

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3


In [10]:
df_raw.loc[20:30]

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
20,2023,94001,Inírida,94,Guainía,3829,Guainía (ETC),9673,77.42,77.33,72.7,73.75,45.35,13.11,89.34,106.92,108.59,82.48,47.78,6.42,6.75,5.08,9.29,3.66,81.12,1.46,81.57,75.11,89.01,12.46,1.46,13.35,15.6,7.33,16.19,8.47,18.36,17.25,6.11,NaN,NaN
21,2023,91798,Tarapacá,91,Amazonas,3828,Amazonas (ETC),1327,50.04,50.04,33.94,62.02,19.96,3.08,55.16,71.56,87.71,35.2,8.37,2.19,0,0.21,8.28,10.53,90.16,0,98.54,67.52,26.32,7.65,0,1.26,24.2,63.16,26.09,1.28,28.87,29.3,31.58,NaN,NaN
22,2023,91669,Puerto Santander,91,Amazonas,3828,Amazonas (ETC),618,63.75,63.75,72.34,78.03,27.1,11.83,69.74,114.89,98.11,41.59,31.18,0.95,1.85,0.39,2.25,0,95.48,0,96.53,89.89,100,3.57,0,3.09,7.87,0,16.67,1.85,19.69,16.85,16.67,NaN,NaN
23,2023,91540,Puerto Nariño,91,Amazonas,3828,Amazonas (ETC),3232,55.07,55.07,37.74,53.98,39.01,16.28,63.18,73.15,73.34,58.83,41.38,9.67,5.85,7.59,12.28,15.43,78.27,0,78.9,72.25,79.26,12.06,0,13.51,15.47,5.32,15.37,5.85,18.61,16.75,3.72,NaN,NaN
24,2023,91536,Puerto Arica,91,Amazonas,3828,Amazonas (ETC),274,83.94,83.94,44.44,87.1,55.95,20.51,95.26,81.48,120.16,79.76,58.97,0,0,0,0,0,99.62,0,99.33,100,100,0.38,0,0.67,0,0,11.49,0,12.75,10.45,17.39,NaN,NaN
25,2023,91530,Puerto Alegría,91,Amazonas,3828,Amazonas (ETC),211,65.4,65.4,40.91,63.64,37.5,0,68.72,77.27,96.97,50,0,1.38,0,0,6.25,0,91.03,0,90.63,87.5,0,7.59,0,9.38,6.25,0,18.62,29.41,19.79,9.38,0,NaN,NaN
26,2023,91460,Miriti - Paraná,91,Amazonas,3828,Amazonas (ETC),571,49.39,49.39,11.63,54.11,27.13,11.01,58.84,39.53,78.35,53.19,34.86,10.42,5.88,2.21,21,23.68,89.58,0,97.79,79,76.32,0,0,0,0,0,12.5,17.65,11.05,18,2.63,NaN,NaN
27,2023,91430,La Victoria,91,Amazonas,3828,Amazonas (ETC),169,14.2,14.2,23.53,15.49,0,0,14.2,29.41,26.76,0,0,0,0,0,0,0,100,0,100,0,0,0,0,0,0,0,16.67,20,15.79,0,0,NaN,NaN
28,2023,91407,La Pedrera,91,Amazonas,3828,Amazonas (ETC),1203,67.17,67.17,55.67,77.6,34.95,19.07,79.97,117.53,100.8,61.17,47.42,0.77,0,0,1.32,6.15,97.14,0,99.4,92.95,89.23,2.09,0,0.6,5.73,4.62,12.97,2.63,20.24,4.85,3.08,NaN,NaN
29,2023,91405,La Chorrera,91,Amazonas,3828,Amazonas (ETC),1197,50.04,50.04,48.75,45.03,33.93,9.9,57.81,67.5,61.35,61.73,35.94,5.38,0,2.45,11.36,4,84.33,5.56,85.93,80,82,10.29,5.56,11.62,8.64,14,26.42,5.56,24.77,37.27,12,NaN,NaN


3. Tomar una muestra aleatoria 

In [11]:
df_raw.sample(frac=0.01, random_state=123456)  # Muestra una muestra aleatoria del 10% de los datos

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
10516,2014,50330,Mesetas,50,Meta,3796,Meta (ETC),2904,76.41,89.01,69.94,74.11,66.11,43.68,85.85,64.17,116.79,75.39,37.5,1.89,0.65,2.27,1.61,0.81,98.11,0,97.73,98.39,99.19,0,0,0,0,0,0,0,0,0,0,25.59,1.75
2152,2022,5628,Sabanalarga,5,Antioquia,3758,Antioquia (ETC),2046,79.33,79.33,50,75.84,72.38,34.94,85.58,66.15,92.17,95.52,58.01,1.8,2.36,1.09,2.78,1.38,97.54,0,98.18,96.35,98.62,0.66,0,0.73,0.87,0,8.32,0,12.39,6.09,1.38,NaN,NaN
9454,2015,41660,Saladoblanco,41,Huila,3790,Huila (ETC),3108,86.87,86.87,70.44,90.16,72.23,30.36,96.59,93.43,113.17,98.53,50,4.3,3.91,3.13,6,5.88,92.62,0,91.31,93.53,93.67,3.07,0,5.56,0.47,0.45,2.68,1.95,3.96,1.29,0.45,33.45,12.5
10130,2014,97889,Yavaraté,97,Vaupés,3831,Vaupés (ETC),386,75.39,87.65,52,70.59,60.76,33.33,86.01,80.65,125.15,60.77,38.71,2.71,NaN,1.96,6.33,NaN,97.29,0,98.04,93.67,100,0,0,0,0,0,3.61,0,4.9,2.53,0,9.76,16.67
11152,2014,5411,Liborina,5,Antioquia,3758,Antioquia (ETC),1985,105.29,89.66,71.79,77.81,68.57,56.41,117.43,109.09,149.4,112.55,68.94,1.53,0.64,0.63,3.19,1.51,98.47,0,99.37,96.81,98.49,0,0,0,0,0,2.87,2.56,2.34,3.62,3.52,27.09,73.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4912,2019,50251,El Castillo,50,Meta,3796,Meta (ETC),1766,95.13,95.13,58.7,94.09,78.35,43.73,102.49,84.78,115.52,106.28,70.85,6.29,3.42,3.58,11.17,4.81,90.2,0,92.25,84.94,93.58,3.52,0,4.17,3.89,1.6,1.44,0,1.19,2.54,0,NaN,NaN
11711,2013,41503,Oporapa,41,Huila,3790,Huila (ETC),3997,68.83,68.8,67.8,77.4,58.5,26,73.9,79.8,88.7,69.3,41.4,3.84,1.51,2.23,7.41,3.17,90.37,0,93.3,83.45,88.69,5.79,0,4.47,9.14,8.14,1.26,0,2.09,0.58,0,NaN,11.11
9773,2015,15897,Zetaquira,15,Boyacá,3769,Boyacá (ETC),981,97.96,97.96,74.7,84.56,100.3,70.07,107.03,81.93,96.44,116.67,129.93,0.99,NaN,0.49,0.8,3.03,96.75,0,99.51,94.68,93.33,2.27,0,0,4.52,3.64,0.79,0,1.48,0.53,0,27.27,0
3428,2020,81065,Arauquita,81,Arauca,3824,Arauca (ETC),13580,68.03,68.03,42.21,67.19,53.53,24.5,76.08,70.52,88.75,75.03,47.58,4.39,3.97,4.22,4.56,5.33,85.71,3.49,87.82,79.93,87.31,9.9,3.49,7.96,15.51,7.36,6.42,1.93,5.15,10.88,2.03,NaN,NaN


In [13]:
df_raw.tail()  # Muestra las últimas 5 filas del DataFrame

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,56.2,82.4,59.9,26.4,82.8,77.4,105,80.4,40.1,8.14,6.6,6.1,13.17,2.44,86.27,0,90.29,77.32,90.24,5.59,0,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,65.1,83.7,62,32.2,91.7,81.1,107.5,89.9,61.4,3.96,3.06,4.3,4.36,1.68,95.46,0,95.12,95.61,95.63,0.58,0,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,61.7,107.2,65.1,27.3,108.4,77.9,146.8,97.3,53.9,8.75,5.83,8.59,9.49,9.74,91.25,0,91.41,90.51,90.26,0,0,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,55.3,76.3,70.7,38.9,87.7,74.3,89.8,96.3,73.2,4.16,2.39,3.23,6.12,3.21,86.86,0.24,90.04,79.42,89.34,8.99,0.24,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33
14584,2011,5021,Alejandría,5,Antioquia,3758,Antioquia (ETC),702,118.52,118.5,82.7,127.6,90.2,48.5,134.9,86.5,159.8,140.8,94,4.03,NaN,NaN,8.46,8.04,92.39,0,97.84,87.15,83.93,3.58,0,2.16,4.39,8.04,0.6,0,0.96,0,0,32.14,81.25


Vamos a crear una copia del conjunto de datos, para cambiar los indices

In [14]:
df_raw2= df_raw.copy()  # Creamos una copia del DataFrame original

In [15]:
df_raw.shape

(14585, 41)

In [16]:
df_raw["new_index"] = range(1, len(df_raw) + 1)  # Añadimos una nueva columna con un índice personalizado

In [19]:
# Creamos una copia del DataFrame y reasignamos los índices para que la primera fila sea 1
df_raw_copia = df_raw.copy()
df_raw_copia.index = range(1, len(df_raw_copia) + 1)
df_raw_copia.head()

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet,new_index
1,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,44.19,63.33,51.53,40.23,66.8,58.14,72.86,66.87,56.32,1.19,0,1.31,0,4.08,92.26,0,96.73,83.49,93.88,6.55,0,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN,1
2,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,33.91,48.89,44.9,21.3,62.98,54.2,65.19,69.6,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0,87.9,84.5,92.98,6.78,0,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN,2
3,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.1,26.16,7.85,15,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.5,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN,3
4,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.9,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN,4
5,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.4,0,89.63,69,78.18,9.24,0,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN,5


Para evaluar si existen registros repetidos, usamos el metodo drop_duplicates

In [20]:
df_raw.drop_duplicates().shape

(14585, 42)

# Conversion de variables, segùn su naturaleza

In [22]:
df_raw['tasa_matriculaci_n_5_16'] = df_raw['tasa_matriculaci_n_5_16'].astype(float)

ahora, vamos a generar una estadistica de resumen a nivel de departmaneots 

In [24]:
dr_raw.groupby(['c_digo_dane_municipio', 'departamento'])['tasa_matriculaci_n_5_16'].mean().reset_index()

NameError: name 'dr_raw' is not defined